In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_core.tools import tool
from langchain.tools.render import render_text_description


@tool
def get_recent_orders_by_status(user, status):
    """
    Fetches the three most recent orders for a given user with a specific status.
    
    Args:
    user (User): The user whose orders are to be retrieved.
    status (str): The status of the orders to be retrieved.

    Returns:
    QuerySet: A list of order objects.
    """

    return f"{user} 사용자의 {status} 상태 주문 조회"

rendered_tools = render_text_description([get_recent_orders_by_status])
print(rendered_tools) 

get_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.

Args:
user (User): The user whose orders are to be retrieved.
status (str): The status of the orders to be retrieved.

Returns:
QuerySet: A list of order objects.


In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.

When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001990E6B9480>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001990E6D3460>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [16]:
base_chain = prompt | model 
base_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

In [13]:
response = base_chain.invoke(
    {"input": "내 이름은 젖수야"}
)
print(response.content)

{
  "necessity": "no"
}


In [14]:
response = base_chain.invoke(
    {"input": "주문 조회를 하고 싶어"}
)
print(response.content)

{
    "necessity": "yes"
}


프롬프트 수정 후 
- When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.

In [17]:
response = base_chain.invoke(
    {"input": "주문 조회를 하고 싶어"}
)
print(response.content)

{
  "necessity": "no"
}


In [ ]:
def determine_function_usage()

In [18]:
general_inquiry_system_prompt = f"""
너는 일반 문의를 처리하는 주문봇이야.

"""

general_inquiry_prompt  = ChatPromptTemplate.from_messages(
    [("system", general_inquiry_system_prompt), ("user", "{input}")]
)


general_inquiry_chain = general_inquiry_prompt | model
general_inquiry_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n너는 일반 문의를 처리하는 주문봇이야.\n\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001990E6B9480>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001990E6D3460>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [19]:
from langchain_core.prompts import ChatPromptTemplate

specific_inquiry_prompt = f"""
You are an order bot that handles inquiries related to customer orders.
You can access the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.
"""

specific_inquiry_prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [20]:
specific_inquiry_chain = specific_inquiry_prompt | model
specific_inquiry_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

----

In [23]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, determine whether there is a necessity to use the tool and respond with yes or no.

When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [26]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class ToolNecessity(BaseModel):
    necessity: str = Field(description="Determines if the use of the tool is necessary.")

    @validator("necessity")
    def validate_necessity(cls, value):
        if value not in ["yes", "no"]:
            raise ValueError("Necessity must be either 'yes' or 'no'.")
        return value
    
parser = PydanticOutputParser(pydantic_object=ToolNecessity)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ToolNecessity'>)

In [27]:
inquiry_classifier_chain1 = prompt | model
inquiry_classifier_chain1

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| Cha

In [28]:
response = inquiry_classifier_chain1.invoke(
    {"input": "주문 조회를 하고 싶어"}
)
print(response.content)

주문 조회를 하려면 어떤 상태의 주문을 조회하고 싶으신가요? 예를 들어, "배송중"이나 "배송완료"와 같은 상태가 있습니다.


In [30]:
inquiry_classifier_chain2 = prompt | model | parser
inquiry_classifier_chain2

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| Cha

In [31]:
response = inquiry_classifier_chain2.invoke(
    {"input": "주문 조회를 하고 싶어"}
)
print(response.content)

OutputParserException: Invalid json output: get_recent_orders_by_status 도구를 사용해야 할 필요가 있습니까?

In [32]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'no'

When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [33]:
inquiry_classifier_chain3 = prompt | model | parser
inquiry_classifier_chain3

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'no'\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'

In [35]:
response = inquiry_classifier_chain2.invoke(
    {"input": "내 이름은 젖수야"}
)
print(response.content)

OutputParserException: Invalid json output: get_recent_orders_by_status 함수를 사용해야 할 필요가 있습니까?

Return your response as a JSON blob with the 'necessity' key. 부분이 있고 없고의 차이가 큰 듯

In [36]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.

When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [37]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019910C63B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000019910D05030>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [38]:
base_chain = prompt | model 
base_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

In [39]:
response = base_chain.invoke(
    {"input": "내 이름은 젖수야"}
)
print(response.content)

{
  "necessity": "no"
}


In [40]:
response = base_chain.invoke(
    {"input": "주문 조회를 하고 싶어"}
)
print(response.content)

{
  "necessity": "no"
}


In [45]:
def route(info):
    print(info)
    if "yes" in info["necessity"].lower():
        return general_inquiry_chain
    else:
        return specific_inquiry_chain

In [46]:
inquiry_chain = base_chain | route
inquiry_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

In [47]:
response = inquiry_chain.invoke(
    {"input": "안녕 나는 '나들'이야"}
)
print(response.content)

content='{\n  "necessity": "no"\n}' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 192, 'total_tokens': 203}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4bc51200-97ee-4215-84a5-8f2e9d49e005-0'


TypeError: 'AIMessage' object is not subscriptable

In [48]:
from langchain_core.messages import AIMessage

def route(AIMessage: AIMessage):
    response_dict = AIMessage.content

    if "yes" in response_dict["necessity"].lower():
        return general_inquiry_chain
    else:
        return specific_inquiry_chain

In [49]:
inquiry_chain = base_chain | route
inquiry_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

json 객체 스타일이지만 문자열

In [52]:
response = base_chain.invoke(
    {"input": "안녕 나는 '나들'이야"}
)
response.content

'{\n  "necessity": "no"\n}'

In [50]:
response = inquiry_chain.invoke(
    {"input": "안녕 나는 '나들'이야"}
)
print(response.content)

TypeError: string indices must be integers

In [53]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_parser = SimpleJsonOutputParser()

base_chain = prompt | model | json_parser
base_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

In [59]:

def route(output):
    print(output)
    if "yes" in output["necessity"].lower():
        return general_inquiry_chain
    else:
        return specific_inquiry_chain

In [60]:
inquiry_chain = base_chain | route
inquiry_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with yes or no. Return your response as a JSON blob with the 'necessity' key.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTe

In [61]:
response = inquiry_chain.invoke(
    {"input": "안녕 나는 '나들'이야"}
)
print(response.content)

{'necessity': 'no'}


KeyError: "Input to ChatPromptTemplate is missing variables {'input'}.  Expected: ['input'] Received: ['necessity']"

In [58]:
response = base_chain.invoke(
    {"input": "안녕 나는 '나들'이야"}
)
print(response)

{'necessity': 'no'}


In [64]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'no'.

When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [65]:
base_chain = prompt | model
base_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'no'.\n\nWhen it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}

In [66]:
base_chain.invoke(
    {"input": "내 이름은 '나들'이라고 해"}
)

AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 179, 'total_tokens': 180}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2bbc246c-283c-4ab4-b553-594733df40c7-0')

In [67]:
base_chain.invoke(
    {"input": "주문 조회 좀 하고 싶어요"}
)

AIMessage(content='어떤 주문 상태를 조회하고 싶으신가요? 현재 주문 상태가 필요하신 경우가 있으면 말씀해주세요.', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 179, 'total_tokens': 221}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7f656143-77e6-4ec7-9259-feb784bf6ed5-0')

In [81]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class ToolNecessity(BaseModel):
    necessity: str = Field(description="Determines if the use of the tool is necessary.")

    @validator("necessity")
    def validate_necessity(cls, value):
        if value not in ["yes", "no"]:
            raise ValueError("Necessity must be either 'yes' or 'no'.")
        return value
    
parser = PydanticOutputParser(pydantic_object=ToolNecessity)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ToolNecessity'>)

In [82]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'

In [83]:
from langchain_core.prompts import PromptTemplate

template = """
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'no'.
When it's possible to determine the required arguments for a function call from the preceding conversation or the user's current input message, only then should the function be considered for use.

\n{format_instructions}\n{query}\n
"""
prompt = PromptTemplate(
    template=template ,
    input_variables=["query", "rendered_tools"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt 

PromptTemplate(input_variables=['query', 'rendered_tools'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'}, template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\n{rendered_tools}\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'n

In [84]:
base_chain = prompt | model
base_chain

PromptTemplate(input_variables=['query', 'rendered_tools'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}\n```'}, template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\n{rendered_tools}\n\nGiven the user input, determine whether there is a necessity to use the tool and respond with either 'yes' or 'n

In [85]:
response = base_chain.invoke(
    {"query": "내 이름은 '나들'이라고 해", 
     "rendered_tools": rendered_tools}
)
response

AIMessage(content='{\n  "necessity": "no"\n}', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 341, 'total_tokens': 352}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa6ad163-3657-4ba5-b74a-152bfda8543d-0')

ToolNecessity(necessity='no')

In [87]:
response = base_chain.invoke(
    {"query": "주문 조회 가능한가요?", 
     "rendered_tools": rendered_tools}
)
parser.invoke(response)

OutputParserException: Failed to parse ToolNecessity from completion {"properties": {"necessity": {"title": "Necessity", "description": "Determines if the use of the tool is necessary.", "type": "string"}}, "required": ["necessity"]}. Got: 1 validation error for ToolNecessity
necessity
  field required (type=value_error.missing)

In [89]:
response = base_chain.invoke(
    {"query": "나는 젖수라고 해", 
     "rendered_tools": rendered_tools}
)
parser.invoke(response)

ToolNecessity(necessity='no')

아래 부분 추가하니 좀 나아진 듯?
- 대답을 출력하기 전에 대답이 'yes' 또는 'no' 중 하나인지 확인하여 반드시 이 둘 중 하나로 응답하게 보장해줘.

In [93]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""
You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.
이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 확인할 수 있을 때만 도구를 사용할 수 있어. 
대답은 반드시 'yes' 또는 'no' 중 하나로 해야해.
대답을 출력하기 전에 대답이 'yes' 또는 'no' 중 하나인지 확인하여 반드시 이 둘 중 하나로 응답하게 보장해줘.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [94]:
base_chain = prompt | model
base_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\nYou are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nget_recent_orders_by_status: get_recent_orders_by_status(user, status) - Fetches the three most recent orders for a given user with a specific status.\n\nArgs:\nuser (User): The user whose orders are to be retrieved.\nstatus (str): The status of the orders to be retrieved.\n\nReturns:\nQuerySet: A list of order objects.\n\n사용자의 입력을 보고 위의 도구를 사용할 필요가 있을지 판단해줘.\n이전 대화 또는 사용자 입력에서 도구 호출에 필요한 인자 모두를 확인할 수 있을 때만 도구를 사용할 수 있어. \n대답은 반드시 'yes' 또는 'no' 중 하나로 해야해.\n대답을 출력하기 전에 대답이 'yes' 또는 'no' 중 하나인지 확인하여 반드시 이 둘 중 하나로 응답하게 보장해줘.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019910C63B50>, async_cli

In [95]:
base_chain.invoke(
    {"input": "내 이름은 '나들'이라고 해"}
)

AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 269, 'total_tokens': 270}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6054a61d-7e8e-4681-b2ef-bcf4791af6c4-0')

In [96]:
base_chain.invoke(
    {"input": "주문 조회 가능한가요?"}
)

AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 266, 'total_tokens': 267}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1a03aa1b-f138-4713-8e26-baeede393ae8-0')

In [97]:
base_chain.invoke(
    {"input": "주문 내역 좀 조회해줘"}
)

AIMessage(content='네, 어떤 상태의 주문을 조회해야 할까요? 현재 주문 상태를 알려주시면 최근 주문 내역을 확인해 드릴 수 있어요.', response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 269, 'total_tokens': 325}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d5da914f-2d3b-4ea3-8d74-21abe367fd0f-0')